In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok
!ngrok authtoken 28Efv8uTnWPnkDv0fJbFQQSjNik_7cPQNgkYrCtfPhb4jpNhC

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=09a3d958ae59a14edabd3dd77569d4a2b19c0346aee9bc95cb77d656595d281d
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import pandas as pd
import random
import numpy as np
from keras.models import Sequential, load_model
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import warnings
from PIL import Image
import requests
from keras.utils import np_utils
from cv2 import cv2
warnings.filterwarnings('ignore')

In [ ]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

In [ ]:
model = load_model("/content/drive/MyDrive/IR Project CODEBASE/weights.h5")

# **Music Recommendation**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/IR Project CODEBASE/Deployment/playlists/Music Playlist.csv')

In [ ]:
data.head()

,Playlist SNo.,Link,Genre,Language
0,1,https://open.spotify.com/playlist/37i9dQZF1DX2...,Rock,Hindi
1,2,https://open.spotify.com/playlist/0kn7JeehPfQx...,Rock,Hindi
2,3,https://open.spotify.com/playlist/0V8rICPS7HsI...,Rock,Hindi
3,4,https://open.spotify.com/playlist/37i9dQZF1DX2...,Rock,Hindi
4,5,https://open.spotify.com/playlist/1WLxavbTX7zs...,Rock,Hindi


In [ ]:
music_playlist = data.iloc[:, :].values

In [ ]:
dic_map = {
    0: ['Instrumental', 'Classical', 'Ballad'],
    1: ['Ballad', 'Instrumental', 'Rock'],
    2: ['EDM', 'Jazz', 'Rock', 'Pop'],
    3: ['Pop', 'Rock', 'EDM', 'Blues', 'Hip Hop'],
    4: ['Classical', 'Instrumental', 'Ballad'],
    5: ['Folk', 'Rock', 'Pop'],
    6: ['EDM', 'Rock', 'Jazz']
}

In [ ]:
def emotion_prediction(model, image):
  # 0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral
  dic_emotion = {0:'angry', 1:'disgust', 2:'fear', 3:'happy', 4:'sad', 5:'surprise', 6:'neutral'}
  # IMAGE =image.reshape((1, 48, 48, 1))
  model_pred = model.predict(image.reshape((-1, 48, 48, 1)))
  print("INSIDEEE",model_pred.shape)
  print("Image_szie",image.shape)

  y_predicted = np.argmax(model_pred)
  # plt.imshow(image)

  print("PREDICTION",y_predicted)
  predicted_emotion = dic_emotion[y_predicted]
  image_intensity = np.max(model_pred)
  final_intensity = image_intensity*100
  # predicted_emotion="happy"
  # image_intensity=0.9
  return predicted_emotion, image_intensity

In [ ]:
def suggest_music(emotion, image_intensity):
  dic_emotion = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'sad':4, 'surprise':5, 'neutral':6}
  emotion_val = dic_emotion[emotion]
  data_list = []
  list_genres = music_playlist[:, 2]
  i = 0
  if image_intensity > 0.32 and image_intensity < 0.63:
    i=0
  elif image_intensity >= 0.63 and image_intensity < 0.83:
    i=1
  else:
    i=2
  i = dic_map[emotion_val][i]
  for j in range(len(list_genres)):
    if list_genres[j] == i:
      data_list.append(music_playlist[j])
  random.shuffle(data_list)

  return data_list[0][1], data_list[0][3], data_list[0][2]

# **Movie Recommendation**

In [ ]:
data_movie = pd.read_csv('/content/drive/MyDrive/IR Project CODEBASE/Deployment/playlists/Movies_PlayList.csv')

In [ ]:
data_movie['Category'].unique()

array(['Animation', 'Drama', 'Epic', 'Fantasy', 'Gangster', 'Mystery',
       'Romantic Comedy', 'Science Fiction', 'Horror', 'Action'],
      dtype=object)

In [ ]:
dic_map_movie = {
    0: ['Drama', 'Romantic Comedy'],
    1: ['Mystery'],
    2: ['Epic'],
    3: ['Drama', 'Romantic Comedy', 'Mystery', 'Epic', 'Animation', 'Action', 'Fantasy', 'Horror', 'Science Fiction', 'Gangster'],
    4: ['Animation', 'Action'],
    5: ['Fantasy', 'Horror'],
    6: ['Science Fiction', 'Gangster']
}

In [ ]:
haar_cascade_face = cv2.CascadeClassifier('/content/drive/MyDrive/IR Project CODEBASE/haarcascade_frontalface_alt2.xml')


In [ ]:
def detect_faces(cascade, test_image, scaleFactor = 1.1, width=5):
    # create a copy of the image to prevent any changes to the original one.
    image_copy = test_image.copy()
    
    #convert the test image to gray scale as opencv face detector expects gray images
    # gray_image = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
    
    # Applying the haar classifier to detect faces
    faces_rect = cascade.detectMultiScale(image_copy, scaleFactor=scaleFactor, minNeighbors=5)
    h1 = 0
    w1 = 0
    for (x, y, w, h) in faces_rect:
        cv2.rectangle(image_copy, (x, y), (x+w, y+h), (255, 255, 0), width)
        h1 = h
        w1 = w
        break
    crop_img = image_copy[y+width:y+h1-width, x+width:x+w1-width]
    
    return crop_img, image_copy

In [ ]:
def suggest_movie(emotion):
    from random import shuffle
    emotions = {'angry':0, 'disgust':1, 'fear':2, 'happy':3, 'sad':4, 'surprise':5, 'neutral':6}
    emotion_label = emotions[emotion]
    list_data = []
    list_cat = []
    list_movie_name = []
    category = []
    category = dic_map_movie[emotion_label]
    for index,row in data_movie.iterrows():
      for i in category:
        if row['Category'] == i:
          list_data.append(row['Link'])
          list_cat.append(row['Category'])
          list_movie_name.append(row['Movie Name'])

    list_data_shuf = []
    list_cat_shuf = []
    list_movie_name_shuf = []
    index_shuf = list(range(len(list_data)))
    shuffle(index_shuf)
    for i in index_shuf:
     list_data_shuf.append(list_data[i])
     list_cat_shuf.append(list_cat[i])
     list_movie_name_shuf.append(list_movie_name[i])
    # random.shuffle(list_data)
    print('We suggest you this movie:', list_data)
    print('Categories recommended: ',category)
    print('For emotion type: ', emotion)

    return list_data_shuf, list_cat_shuf, list_movie_name_shuf

# **Deployment Using Flask**

In [ ]:
from flask_ngrok import run_with_ngrok

UPLOAD_FOLDER = '/content'

app = Flask(__name__, template_folder='/content/drive/MyDrive/IR Project CODEBASE/Deployment/templates', static_folder='/content/drive/MyDrive/IR Project CODEBASE/Deployment/static')
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

run_with_ngrok(app)

  
@app.route("/",methods=['POST','GET'])
def home():
  return render_template('GettingStarted.html')


@app.route('/recommendations',methods=['POST','GET'])
def recommendations():

  img = request.files['file1']
  img.save('capturedImg.jpg')
  img = cv2.imread('capturedImg.jpg')
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  faces, original = detect_faces(haar_cascade_face, gray, width=1)
  # plt.imshow(faces)
  res = cv2.resize(faces, dsize=(48,48), interpolation=cv2.INTER_CUBIC)
  res1 = res.reshape(-1, 48, 48, 1)
  # print(res1.shape)

  emotion, image_intensity = emotion_prediction(model, res1)
  # plt.imshow(res1.reshape(48, 48))
  # emotion="happy"
  # image_intensity=0.9

  music_playlist, language, genre = suggest_music(emotion, image_intensity)
  movie_links, movie_categories, movie_names = suggest_movie(emotion)
  return render_template('Recommendations.html', prediction_text = emotion, music_list = music_playlist, music_lang = language, music_genre = genre, moviesLinks = movie_links, moviesCat = movie_categories, moviesNames = movie_names)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fb9d-34-147-105-206.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [27/Apr/2022 16:44:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:44:24] "GET /static/123.jpg HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:44:24] "GET /static/logo.png HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:44:26] "GET /favicon.ico HTTP/1.1" 404 -


INSIDEEE (1, 7)
Image_szie (1, 48, 48, 1)
PREDICTION 3
We suggest you this movie: ['https://www.imdb.com/title/tt0245429/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0110357/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0095327/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt5311514/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt4633694/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt2380307/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0910970/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0119698/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt1049413/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0435761/?ref_=adv_li_tt', 'https://www.imdb.com/title/tt0056592/?ref_=ttls_li_tt', 'https://www.imdb.com/title/tt0117381/?ref_=ttls_li_tt', 'https://www.imdb.com/title/tt0117293/?ref_=ttls_li_tt', 'https://www.imdb.com/title/tt0050083/?ref_=ttls_li_tt', 'https://www.imdb.com/title/tt0195685/?ref_=ttls_li_tt', 'https://www.imdb.com/title/tt0404032/?ref_=ttls_li_tt', 'https://www.im

127.0.0.1 - - [27/Apr/2022 16:44:36] "POST /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2022 16:45:56] "GET / HTTP/1.1" 200 -
